# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [126]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

import re
import numpy as np
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import pos_tag, ne_chunk

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sqlalchemy import create_engine

import warnings 
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mintk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mintk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mintk\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mintk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [60]:
# load data from database
engine = create_engine('sqlite:///Disaster.db')
df = pd.read_sql_table('messages', engine)
X = df['message']
Y = df.iloc[:, 4:]

In [61]:
# Print the first 5 rows of X in full
for i in range(5):
    print(X[i])

Weather update - a cold front from Cuba that could pass over Haiti
Is the Hurricane over or is it not over
Looking for someone but no name
UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
says: west side of Haiti, rest of the country today and tonight


### 2. Write a tokenization function to process your text data

In [38]:
def tokenize(text):
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    # Tokenize text
    tokens = word_tokenize(text)

    # Lemmatize and remove stop words
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words("english")
    clean_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return clean_tokens

In [71]:
tokenize(X[3])

['un',
 'report',
 'leogane',
 '80',
 '90',
 'destroyed',
 'hospital',
 'st',
 'croix',
 'functioning',
 'need',
 'supply',
 'desperately']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [52]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(LogisticRegression()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [90]:
# Find unique values for columns
one_class = []
for col in Y.columns:
    # print(col, np.unique(Y[col]))
    if len(np.unique(Y[col])) < 2:
        print(col, np.unique(Y[col]))
        one_class.append(col)

# Drop columns with only one class
Y.drop(one_class, axis=1, inplace=True) #This is done so that the model does not break

child_alone [0]


In [91]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [92]:
# Train pipeline
pipeline.fit(X_train, y_train)

c:\Users\mintk\Desktop\Udacity\disaster_response_pipeline_project\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\mintk\Desktop\Udacity\disaster_response_pipeline_project\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000002470082DEE0>)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=LogisticRegression()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [95]:
# Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.
y_pred = pipeline.predict(X_test)
for i, col in enumerate(y_test.columns):
    print(col)
    print(classification_report(y_test[col], y_pred[:, i]))

related
              precision    recall  f1-score   support

           0       0.73      0.48      0.58      1495
           1       0.85      0.95      0.90      5014
           2       1.00      0.02      0.04        45

    accuracy                           0.83      6554
   macro avg       0.86      0.48      0.51      6554
weighted avg       0.83      0.83      0.82      6554

request
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      5435
           1       0.80      0.52      0.63      1119

    accuracy                           0.89      6554
   macro avg       0.85      0.75      0.78      6554
weighted avg       0.89      0.89      0.89      6554

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6515
           1       0.00      0.00      0.00        39

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      655

c:\Users\mintk\Desktop\Udacity\disaster_response_pipeline_project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mintk\Desktop\Udacity\disaster_response_pipeline_project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mintk\Desktop\Udacity\disaster_response_pipeline_project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

              precision    recall  f1-score   support

           0       0.95      0.99      0.97      6010
           1       0.84      0.42      0.56       544

    accuracy                           0.95      6554
   macro avg       0.89      0.71      0.76      6554
weighted avg       0.94      0.95      0.94      6554

clothing
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6459
           1       0.67      0.17      0.27        95

    accuracy                           0.99      6554
   macro avg       0.83      0.58      0.63      6554
weighted avg       0.98      0.99      0.98      6554

money
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6399
           1       0.75      0.08      0.14       155

    accuracy                           0.98      6554
   macro avg       0.86      0.54      0.56      6554
weighted avg       0.97      0.98      0.97      6554

miss

c:\Users\mintk\Desktop\Udacity\disaster_response_pipeline_project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mintk\Desktop\Udacity\disaster_response_pipeline_project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mintk\Desktop\Udacity\disaster_response_pipeline_project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6485
           1       0.00      0.00      0.00        69

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

shops
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6524
           1       0.00      0.00      0.00        30

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554

aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6474
           1       0.00      0.00      0.00        80

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

o

c:\Users\mintk\Desktop\Udacity\disaster_response_pipeline_project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mintk\Desktop\Udacity\disaster_response_pipeline_project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mintk\Desktop\Udacity\disaster_response_pipeline_project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

              precision    recall  f1-score   support

           0       0.88      0.96      0.92      5308
           1       0.75      0.44      0.56      1246

    accuracy                           0.87      6554
   macro avg       0.81      0.70      0.74      6554
weighted avg       0.86      0.87      0.85      6554



### 6. Improve your model
Use grid search to find better parameters. 

In [96]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x000002470082DEE0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=LogisticRegression()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x000002470082DEE0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=LogisticRegression()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf_

In [118]:
parameters =  {
    'vect__max_df': (0.5, 0.75, 1.0), # max_df=0.5 means "ignore terms that appear in more than 50% of the documents"
    # 'vect__ngram_range': ((1, 1), (1, 2)), # unigrams or bigrams
    'clf__estimator__C': (0.1, 1, 10, 100), # Regularization parameter
    # 'clf__estimator__penalty': ['l1', 'l2'] # Norm used in penalization; l1 is the sum of the absolute value of the coefficients; l2 is the sum of the squared coefficients
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=2) # reduced cv to 2 to speed up training

In [119]:
# Train pipeline
cv.fit(X_train, y_train) # vect__max_df and clf__estimator__C take about 10 minutes to train with cv=2

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000002470082DEE0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=LogisticRegression()))]),
             param_grid={'clf__estimator__C': (0.1, 1, 10, 100),
                         'vect__max_df': (0.5, 0.75, 1.0)})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [120]:
# Test model
y_pred = cv.predict(X_test)
# accuracy = (y_pred == y_test).mean()
# print("\nAccuracy:", accuracy)
print("Best Parameters:", cv.best_params_)

Best Parameters: {'clf__estimator__C': 10, 'vect__max_df': 0.5}

Accuracy: related                   0.816906
request                   0.890601
offer                     0.994049
aid_related               0.761367
medical_help              0.921422
medical_products          0.955142
search_and_rescue         0.975740
security                  0.980165
military                  0.970247
water                     0.963381
food                      0.948276
shelter                   0.946750
clothing                  0.988404
money                     0.978639
missing_people            0.989014
refugees                  0.970400
death                     0.965060
other_aid                 0.863747
infrastructure_related    0.932408
transport                 0.956210
buildings                 0.954226
electricity               0.981691
tools                     0.994812
hospitals                 0.989014
shops                     0.995423
aid_centers               0.987794
other_infrastru

In [121]:
# Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.
for i, col in enumerate(y_test.columns):
    print(col)
    print(classification_report(y_test[col], y_pred[:, i]))

related
              precision    recall  f1-score   support

           0       0.62      0.54      0.58      1495
           1       0.86      0.90      0.88      5014
           2       0.71      0.22      0.34        45

    accuracy                           0.82      6554
   macro avg       0.73      0.56      0.60      6554
weighted avg       0.81      0.82      0.81      6554

request
              precision    recall  f1-score   support

           0       0.91      0.96      0.94      5435
           1       0.73      0.56      0.64      1119

    accuracy                           0.89      6554
   macro avg       0.82      0.76      0.79      6554
weighted avg       0.88      0.89      0.88      6554

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6515
           1       0.00      0.00      0.00        39

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      655

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [122]:
# A custom transformer that extracts the length of the text data
class TextLengthExtractor(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, X):
        return pd.DataFrame(X.apply(lambda x: len(x)))

In [127]:
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('text_length', TextLengthExtractor()) # Added new feature which is the length of the text
    ])),

    ('clf', MultiOutputClassifier(SGDClassifier())) # Changed to SGDClassifier because it is faster than LogisticRegression
])

In [128]:
# Train pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x000002470082DEE0>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('text_length',
                                                 TextLengthExtractor())])),
                ('clf', MultiOutputClassifier(estimator=SGDClassifier()))])

In [133]:
# Test model
y_pred = pipeline.predict(X_test)
# accuracy = (y_pred == y_test).mean()
# print("\nAccuracy:", accuracy)

for i, col in enumerate(y_test.columns):
    print(col)
    print(classification_report(y_test[col], y_pred[:, i]))

related
              precision    recall  f1-score   support

           0       0.59      0.26      0.36      1495
           1       0.81      0.95      0.87      5014
           2       0.00      0.00      0.00        45

    accuracy                           0.78      6554
   macro avg       0.46      0.40      0.41      6554
weighted avg       0.75      0.78      0.75      6554

request
              precision    recall  f1-score   support

           0       0.89      0.90      0.89      5435
           1       0.48      0.45      0.46      1119

    accuracy                           0.82      6554
   macro avg       0.68      0.68      0.68      6554
weighted avg       0.82      0.82      0.82      6554

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6515
           1       0.00      0.00      0.00        39

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      655

### 9. Export your model as a pickle file

In [ ]:
# Export model as a pickle file
import pickle
pickle.dump(pipeline, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.